# Project: Train a Quadcopter How to Fly

Design an agent to fly a quadcopter, and then train it using a reinforcement learning algorithm of your choice! 

Try to apply the techniques you have learnt, but also feel free to come up with innovative ideas and test them.

## Instructions

Take a look at the files in the directory to better understand the structure of the project. 

- `task.py`: Define your task (environment) in this file.
- `agents/`: Folder containing reinforcement learning agents.
    - `policy_search.py`: A sample agent has been provided here.
    - `agent.py`: Develop your agent here.
- `physics_sim.py`: This file contains the simulator for the quadcopter.  **DO NOT MODIFY THIS FILE**.

For this project, you will define your own task in `task.py`.  Although we have provided a example task to get you started, you are encouraged to change it.  Later in this notebook, you will learn more about how to amend this file.

You will also design a reinforcement learning agent in `agent.py` to complete your chosen task.  

You are welcome to create any additional files to help you to organize your code.  For instance, you may find it useful to define a `model.py` file defining any needed neural network architectures.

## Controlling the Quadcopter

We provide a sample agent in the code cell below to show you how to use the sim to control the quadcopter.  This agent is even simpler than the sample agent that you'll examine (in `agents/policy_search.py`) later in this notebook!

The agent controls the quadcopter by setting the revolutions per second on each of its four rotors.  The provided agent in the `Basic_Agent` class below always selects a random action for each of the four rotors.  These four speeds are returned by the `act` method as a list of four floating-point numbers.  

For this project, the agent that you will implement in `agents/agent.py` will have a far more intelligent method for selecting actions!

In [ ]:
import random

class Basic_Agent():
    def __init__(self, task):
        self.task = task
    
    def act(self):
        new_thrust = random.gauss(450., 25.)
        return [new_thrust + random.gauss(0., 1.) for x in range(4)]

Run the code cell below to have the agent select actions to control the quadcopter.  

Feel free to change the provided values of `runtime`, `init_pose`, `init_velocities`, and `init_angle_velocities` below to change the starting conditions of the quadcopter.

The `labels` list below annotates statistics that are saved while running the simulation.  All of this information is saved in a text file `data.txt` and stored in the dictionary `results`.  

In [ ]:
%load_ext autoreload
%autoreload 2

import csv
import numpy as np
from task import Task

# Modify the values below to give the quadcopter a different starting position.
runtime = 5.                                     # time limit of the episode
init_pose = np.array([0., 0., 10., 0., 0., 0.])  # initial pose
init_velocities = np.array([0., 0., 0.])         # initial velocities
init_angle_velocities = np.array([0., 0., 0.])   # initial angle velocities
file_output = 'data.txt'                         # file name for saved results

# Setup
task = Task(init_pose, init_velocities, init_angle_velocities, runtime)
agent = Basic_Agent(task)
done = False
labels = ['time', 'x', 'y', 'z', 'phi', 'theta', 'psi', 'x_velocity',
          'y_velocity', 'z_velocity', 'phi_velocity', 'theta_velocity',
          'psi_velocity', 'rotor_speed1', 'rotor_speed2', 'rotor_speed3', 'rotor_speed4']
results = {x : [] for x in labels}

# Run the simulation, and save the results.
with open(file_output, 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(labels)
    while True:
        rotor_speeds = agent.act()
        _, _, done = task.step(rotor_speeds)
        to_write = [task.sim.time] + list(task.sim.pose) + list(task.sim.v) + list(task.sim.angular_v) + list(rotor_speeds)
        for ii in range(len(labels)):
            results[labels[ii]].append(to_write[ii])
        writer.writerow(to_write)
        if done:
            break

Run the code cell below to visualize how the position of the quadcopter evolved during the simulation.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(results['time'], results['x'], label='x')
plt.plot(results['time'], results['y'], label='y')
plt.plot(results['time'], results['z'], label='z')
plt.legend()
_ = plt.ylim()

The next code cell visualizes the velocity of the quadcopter.

In [ ]:
plt.plot(results['time'], results['x_velocity'], label='x_hat')
plt.plot(results['time'], results['y_velocity'], label='y_hat')
plt.plot(results['time'], results['z_velocity'], label='z_hat')
plt.legend()
_ = plt.ylim()

Next, you can plot the Euler angles (the rotation of the quadcopter over the $x$-, $y$-, and $z$-axes),

In [ ]:
plt.plot(results['time'], results['phi'], label='phi')
plt.plot(results['time'], results['theta'], label='theta')
plt.plot(results['time'], results['psi'], label='psi')
plt.legend()
_ = plt.ylim()

before plotting the velocities (in radians per second) corresponding to each of the Euler angles.

In [ ]:
plt.plot(results['time'], results['phi_velocity'], label='phi_velocity')
plt.plot(results['time'], results['theta_velocity'], label='theta_velocity')
plt.plot(results['time'], results['psi_velocity'], label='psi_velocity')
plt.legend()
_ = plt.ylim()

Finally, you can use the code cell below to print the agent's choice of actions.  

In [ ]:
plt.plot(results['time'], results['rotor_speed1'], label='Rotor 1 revolutions / second')
plt.plot(results['time'], results['rotor_speed2'], label='Rotor 2 revolutions / second')
plt.plot(results['time'], results['rotor_speed3'], label='Rotor 3 revolutions / second')
plt.plot(results['time'], results['rotor_speed4'], label='Rotor 4 revolutions / second')
plt.legend()
_ = plt.ylim()

When specifying a task, you will derive the environment state from the simulator.  Run the code cell below to print the values of the following variables at the end of the simulation:
- `task.sim.pose` (the position of the quadcopter in ($x,y,z$) dimensions and the Euler angles),
- `task.sim.v` (the velocity of the quadcopter in ($x,y,z$) dimensions), and
- `task.sim.angular_v` (radians/second for each of the three Euler angles).

In [ ]:
# the pose, velocity, and angular velocity of the quadcopter at the end of the episode
print(task.sim.pose)
print(task.sim.v)
print(task.sim.angular_v)

In the sample task in `task.py`, we use the 6-dimensional pose of the quadcopter to construct the state of the environment at each timestep.  However, when amending the task for your purposes, you are welcome to expand the size of the state vector by including the velocity information.  You can use any combination of the pose, velocity, and angular velocity - feel free to tinker here, and construct the state to suit your task.

## The Task

A sample task has been provided for you in `task.py`.  Open this file in a new window now. 

The `__init__()` method is used to initialize several variables that are needed to specify the task.  
- The simulator is initialized as an instance of the `PhysicsSim` class (from `physics_sim.py`).  
- Inspired by the methodology in the original DDPG paper, we make use of action repeats.  For each timestep of the agent, we step the simulation `action_repeats` timesteps.  If you are not familiar with action repeats, please read the **Results** section in [the DDPG paper](https://arxiv.org/abs/1509.02971).
- We set the number of elements in the state vector.  For the sample task, we only work with the 6-dimensional pose information.  To set the size of the state (`state_size`), we must take action repeats into account.  
- The environment will always have a 4-dimensional action space, with one entry for each rotor (`action_size=4`). You can set the minimum (`action_low`) and maximum (`action_high`) values of each entry here.
- The sample task in this provided file is for the agent to reach a target position.  We specify that target position as a variable.

The `reset()` method resets the simulator.  The agent should call this method every time the episode ends.  You can see an example of this in the code cell below.

The `step()` method is perhaps the most important.  It accepts the agent's choice of action `rotor_speeds`, which is used to prepare the next state to pass on to the agent.  Then, the reward is computed from `get_reward()`.  The episode is considered done if the time limit has been exceeded, or the quadcopter has travelled outside of the bounds of the simulation.

In the next section, you will learn how to test the performance of an agent on this task.

## The Agent

The sample agent given in `agents/policy_search.py` uses a very simplistic linear policy to directly compute the action vector as a dot product of the state vector and a matrix of weights. Then, it randomly perturbs the parameters by adding some Gaussian noise, to produce a different policy. Based on the average reward obtained in each episode (`score`), it keeps track of the best set of parameters found so far, how the score is changing, and accordingly tweaks a scaling factor to widen or tighten the noise.

Run the code cell below to see how the agent performs on the sample task.

In [ ]:
import sys
import pandas as pd
from agents.policy_search import PolicySearch_Agent
from task import Task

num_episodes = 1000
target_pos = np.array([0., 0., 10.])
task = Task(target_pos=target_pos)
agent = PolicySearch_Agent(task) 

for i_episode in range(1, num_episodes+1):
    state = agent.reset_episode() # start a new episode
    while True:
        action = agent.act(state) 
        next_state, reward, done = task.step(action)
        agent.step(reward, done)
        state = next_state
        if done:
            print("\rEpisode = {:4d}, score = {:7.3f} (best = {:7.3f}), noise_scale = {}".format(
                i_episode, agent.score, agent.best_score, agent.noise_scale), end="")  # [debug]
            break
    sys.stdout.flush()

This agent should perform very poorly on this task.  And that's where you come in!

## Define the Task, Design the Agent, and Train Your Agent!

Amend `task.py` to specify a task of your choosing.  If you're unsure what kind of task to specify, you may like to teach your quadcopter to takeoff, hover in place, land softly, or reach a target pose.  

After specifying your task, use the sample agent in `agents/policy_search.py` as a template to define your own agent in `agents/agent.py`.  You can borrow whatever you need from the sample agent, including ideas on how you might modularize your code (using helper methods like `act()`, `learn()`, `reset_episode()`, etc.).

Note that it is **highly unlikely** that the first agent and task that you specify will learn well.  You will likely have to tweak various hyperparameters and the reward function for your task until you arrive at reasonably good behavior.

As you develop your agent, it's important to keep an eye on how it's performing. Use the code above as inspiration to build in a mechanism to log/save the total rewards obtained in each episode to file.  If the episode rewards are gradually increasing, this is an indication that your agent is learning.

In [ ]:
## TODO: Train your agent here.
import numpy as np
import sys
from agents.agent import DDPG
from takeoff import Takeoff
#from hover import Hover



num_episodes = 1000
init_pose = np.array([0.0, 0.0, 1.0, 0.0, 0.0, 0.0])
target_pos = np.array([0., 0., 20.])
task = Takeoff(init_pose=init_pose,target_pos=target_pos)
agent = DDPG(task) 
avg_rewards=[]
scores=[]
result_x=[]
result_y=[]
result_z=[]
for i_episode in range(1, num_episodes+1):
    state = agent.reset_episode() # start a new episode
    score = 0
    count = 0
    while True:
        action = agent.act(state) 
        next_state, reward, done = agent.task.step(action)
        agent.step(action, reward, next_state, done)
        state = next_state
        score += reward
        count +=1
        if done:
            break
        avg_reward = score / count    
    avg_rewards.append(avg_reward)
    scores.append(score)
    result_x.append(task.sim.pose[0])
    result_y.append(task.sim.pose[1])
    result_z.append(task.sim.pose[2])
    print("\rEpisode = {:4d}/{}".format(i_episode,num_episodes))
    print(task.sim.pose[2])
    if i_episode % 10 ==0:
        print("average reward: {}".format(np.mean(avg_rewards[-10:])))
    sys.stdout.flush()


C:\Users\Han\Anaconda3\envs\reinforce\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Episode =    1/1000
0.0
Episode =    2/1000
0.0
Episode =    3/1000
0.0
Episode =    4/1000
0.0
Episode =    5/1000
0.0
Episode =    6/1000
0.0
Episode =    7/1000
0.0
Episode =    8/1000
0.0
Episode =    9/1000
0.0
Episode =   10/1000
0.0
average reward: 10.197907716797001
Episode =   11/1000
0.0
Episode =   12/1000
0.0
Episode =   13/1000
0.0
Episode =   14/1000
0.0
Episode =   15/1000
0.0
Episode =   16/1000
0.0
Episode =   17/1000
0.0
Episode =   18/1000
0.0
Episode =   19/1000
0.0
Episode =   20/1000
0.0
average reward: 6.465378709303233
Episode =   21/1000
0.0
Episode =   22/1000
0.0
Episode =   23/1000
0.0
Episode =   24/1000
0.0
Episode =   25/1000
0.0
Episode =   26/1000
0.0
Episode =   27/1000
0.0
Episode =   28/1000
0.0
Episode =   29/1000
0.0
Episode =   30/1000
0.0
average reward: 6.235122456411581
Episode =   31/1000
0.0
Episode =   32/1000
0.0
Episode =   33/1000
0.0
Episode =   34/1000
0.0
Episode =   35/1000
0.0
Episode =   36/1000
0.0
Episode =   37/1000
0.0
Episode =

Episode =  300/1000
0.0
average reward: 5.618177316206783
Episode =  301/1000
0.0
Episode =  302/1000
0.0
Episode =  303/1000
0.0
Episode =  304/1000
0.0
Episode =  305/1000
0.0
Episode =  306/1000
0.0
Episode =  307/1000
0.0
Episode =  308/1000
0.0
Episode =  309/1000
0.0
Episode =  310/1000
0.0
average reward: 5.581202285361887
Episode =  311/1000
0.0
Episode =  312/1000
0.0
Episode =  313/1000
0.0
Episode =  314/1000
0.0
Episode =  315/1000
0.0
Episode =  316/1000
0.0
Episode =  317/1000
0.0
Episode =  318/1000
0.0
Episode =  319/1000
0.0
Episode =  320/1000
0.0
average reward: 5.559182534907579
Episode =  321/1000
0.0
Episode =  322/1000
0.0
Episode =  323/1000
0.0
Episode =  324/1000
0.0
Episode =  325/1000
0.0
Episode =  326/1000
0.0
Episode =  327/1000
0.0
Episode =  328/1000
0.0
Episode =  329/1000
0.0
Episode =  330/1000
0.0
average reward: 5.615986898016819
Episode =  331/1000
0.0
Episode =  332/1000
0.0
Episode =  333/1000
0.0
Episode =  334/1000
0.0
Episode =  335/1000
0.0


Episode =  600/1000
0.0
average reward: 5.300293637085063
Episode =  601/1000
0.0
Episode =  602/1000
0.0
Episode =  603/1000
0.0
Episode =  604/1000
0.0
Episode =  605/1000
0.0
Episode =  606/1000
0.0
Episode =  607/1000
0.0
Episode =  608/1000
0.0
Episode =  609/1000
0.0
Episode =  610/1000
0.0
average reward: 5.1886017725256846
Episode =  611/1000
0.0
Episode =  612/1000
0.0
Episode =  613/1000
0.0
Episode =  614/1000
0.0
Episode =  615/1000
0.0
Episode =  616/1000
0.0
Episode =  617/1000
0.0
Episode =  618/1000
0.0
Episode =  619/1000
0.0
Episode =  620/1000
0.0
average reward: 4.919678321656803
Episode =  621/1000
0.0
Episode =  622/1000
0.0
Episode =  623/1000
0.0
Episode =  624/1000
0.0
Episode =  625/1000
0.0
Episode =  626/1000
0.0
Episode =  627/1000
0.0
Episode =  628/1000
0.0
Episode =  629/1000
0.0
Episode =  630/1000
0.0
average reward: 4.681574960637984
Episode =  631/1000
0.0
Episode =  632/1000
0.0
Episode =  633/1000
0.0
Episode =  634/1000
0.0
Episode =  635/1000
0.0

## Plot the Rewards

Once you are satisfied with your performance, plot the episode rewards, either from a single run, or averaged over multiple runs. 

In [ ]:
## TODO: Plot the rewards.
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(avg_rewards, label='x')

In [ ]:
plt.plot(scores, label='x')

In [ ]:
plt.plot(result_x, label='x')
plt.plot(result_y, label='y')
plt.plot(result_z, label='z')
plt.legend()

## Reflections

**Question 1**: Describe the task that you specified in `task.py`.  How did you design the reward function?

**Answer**: I wanted to make quadcopter learn 'take off' movement. 

- First, I give reward by its Z position. Second, I give penalty for velocity along x, y axis because I want to make drone move only vertically. and I also give panelty for angular velocity because I think ideal movement without tilting. Third, I give +5 reward if drone did not collipse. Last, I give +20 reward if drone is closer more than half, and I also give +100 reward if drone is located higer position than target and that case I set done = True.

- Here is my code
```
    def get_reward(self):
        reward = 3*self.sim.pose[2]
        reward -= 0.005*((self.sim.v[0])**2+(self.sim.v[1])**2)
        reward -= 0.1*(abs(self.sim.angular_v).sum())
        reward += 5
        if self.sim.v[2]>0:
            reward += 5
        if abs(self.sim.pose[2]-self.target_pos[2])<(self.sim.pose[2]-self.target_pos[2])/2:
            reward += 20
        if self.sim.poseg[2]>self.target_pos[2]:
            reward +=100
        return reward

    def step(self, rotor_speeds):
        """Uses action to obtain next state, reward, done."""
        reward = 0
        pose_all = []
        for _ in range(self.action_repeat):
            done = self.sim.next_timestep(rotor_speeds) # update the sim pose and velocities
            reward = self.get_reward() 
            pose_all.append(self.sim.pose)
            if self.sim.pose[2]>self.target_pos[2]:
                done = True
        next_state = np.concatenate(pose_all)
        return next_state, reward, done
```

**Question 2**: Discuss your agent briefly, using the following questions as a guide:

- What learning algorithm(s) did you try? What worked best for you?
- What was your final choice of hyperparameters (such as $\alpha$, $\gamma$, $\epsilon$, etc.)?
- What neural network architecture did you use (if any)? Specify layers, sizes, activation functions, etc.

**Answer**: I chose most of hyper parameters from *'Lillicrap, Timothy P., et al., 2015. Continuous Control with Deep Reinforcement Learning.'*. However I did not designate learning rate of actor and critor. I set learning rate as paper but it was bad... I also put batchnormalization and regularizer, and change activation function as leaky relu

- here is my actor neural network
```
 net = layers.Dense(units=400, kernel_regularizer = regularizers.l2(0.00001))(states)
        net = layers.BatchNormalization()(net)
        net = layers.LeakyReLU(0.01)(net)
        net = layers.Dense(units=300, kernel_regularizer = regularizers.l2(0.00001))(net)
        net = layers.BatchNormalization()(net)
        net = layers.LeakyReLU(0.01)(net)
        net = layers.Dense(units=200, kernel_regularizer = regularizers.l2(0.00001))(net)
        net = layers.BatchNormalization()(net)
        net = layers.LeakyReLU(0.01)(net)
```
- here is my critor neural network

```
        # Add hidden layer(s) for state pathway
        net_states = layers.Dense(units=300,kernel_regularizer = regularizers.l2(0.00001))(states)
        net_states = layers.BatchNormalization()(net_states)
        net_states = layers.LeakyReLU(0.01)(net_states)
        net_states = layers.Dense(units=400, kernel_regularizer = regularizers.l2(0.00001))(net_states)
        net_states = layers.BatchNormalization()(net_states)
        net_states = layers.LeakyReLU(0.01)(net_states)

        # Add hidden layer(s) for action pathway
        net_actions = layers.Dense(units=400, kernel_regularizer = regularizers.l2(0.00001))(actions)
        net_actions = layers.BatchNormalization()(net_actions)
        net_actions = layers.LeakyReLU(0.01)(net_actions)


        # Try different layer sizes, activations, add batch normalization, regularizers, etc.
        
        # Combine state and action pathways
        net = layers.Add()([net_states, net_actions])
        net = layers.Activation('relu')(net)

        # Add more layers to the combined network if needed
        net = layers.Dense(units=256, kernel_regularizer = regularizers.l2(0.00001))(net)
        net = layers.BatchNormalization()(net)
        net = layers.Activation(activation ='relu')(net)
        # Add final output layer to prduce action values (Q values)
        Q_values = layers.Dense(units=1, activation = None, name='q_values', kernel_initializer=initializers.RandomUniform(minval=-3e-4, maxval=3e-4))(net)

        # Create Keras model
        self.model = models.Model(inputs=[states, actions], outputs=Q_values)
```

- I also set weights as random uniform distribute.

**Question 3**: Using the episode rewards plot, discuss how the agent learned over time.

- Was it an easy task to learn or hard?
- Was there a gradual learning curve, or an aha moment?
- How good was the final performance of the agent? (e.g. mean rewards over the last 10 episodes)

**Answer**:

- set the reward is very hard to me. because the only way to judge it is just run the code.
- I think there is an aha moment in my test
- as look at about x,y,z position. I can see in the last part, drone is almost located at target position. :) 

**Question 4**: Briefly summarize your experience working on this project. You can use the following prompts for ideas.

- What was the hardest part of the project? (e.g. getting started, plotting, specifying the task, etc.)
- Did you find anything interesting in how the quadcopter or your agent behaved?

**Answer**:

- I was very fun.!!! I spent almost 4 days for this project and my real world project is almost broken......... (Professor please forgive me,,,)
- setting the reward function is very hard for me,,,
- It also help review about Keras!(I almost forgot how to use it, I had to look documentation a lot of time)
- It also awesome, when I told to my friends 'this is artificial intelligence about drone controlling!!!' (Thank you)